In [1]:
import sys
sys.path.append("../")
import tensorflow as tf
from utils import dataset_util
from vatic import VaticData, IMDBGroup
import os
from scipy.misc import imread
import sys
from utils import label_map_util
import cv2
from collections import defaultdict
from utils import label_map_util
from coco import coco

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#prepare CLS_mapper
class DefaultClassMapper(defaultdict):
    
    def has_key(self,key):
        return True
    def __contains__(sefl, key):
        return True



CLS_mapper = DefaultClassMapper(lambda: "__background__")

#The target classes
CLS_mapper['bus'] = 'bus'
CLS_mapper['motorcycle'] = "motorcycle"
CLS_mapper['car'] = 'car'
CLS_mapper['bicycle'] = 'bicycle'
CLS_mapper['person'] = 'person'
CLS_mapper['dog'] = 'dog'
CLS_mapper['cat'] = 'cat'
CLS_mapper['truck'] = 'truck'
CLS_mapper['suitcase'] = 'suitcase'
#The classes mapping
CLS_mapper[u'trailer-head'] = "truck"
CLS_mapper[u'sedan/suv'] = "car"
CLS_mapper[u'van'] = "car"
CLS_mapper[u'bike'] = "bicycle"
CLS_mapper['luggage'] = 'suitcase'

In [3]:
coco_train = coco("train", "2014", CLS_mapper=CLS_mapper)   
coco_val = coco("val", "2014", CLS_mapper=CLS_mapper)   
target_vatic_data = ["A1HighwayDay", "airport", "airport2", "B2HighwayNight", "BigCity",\
                     "luggage", "luggage2", "luggage3", "NYC", "Sedans_1", "Sedans_2", \
                     "Thailand-long", "van", "van2", "YuDa"]
vatic_groups = [VaticData(vatic_name, CLS_mapper=CLS_mapper) for vatic_name in target_vatic_data]
imdb_group = IMDBGroup([coco_train, coco_val] + vatic_groups)

loading annotations into memory...
Done (t=13.34s)
creating index...
index created!
loading annotations into memory...
Done (t=6.34s)
creating index...
index created!
Meta data path: /root/data/data-A1HighwayDay/meta.json does not exist. Use Default meta data
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 3160 images
Meta data path: /root/data/data-airport/meta.json does not exist. Use Default meta data
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 604 images
Meta data path: /root/data/data-airport2/meta.json does not exist. Use Default meta data
{'test': {'start': None, 'sets': [1], 'end': None, 'stride': 30}, 'train': {'start': None, 'sets': [0], 'end': None, 'stride': 1}, 'format': 'jpg'}
Total: 1900 images
Meta data path: /root/data/data-B2HighwayNight/meta.json does n

In [4]:
imdb_group.report()

{'__background__': 522586,
 'bicycle': 7370,
 'bus': 6344,
 'car': 45451,
 'cat': 4968,
 'dog': 5718,
 'motorcycle': 9021,
 'person': 280359,
 'suitcase': 6411,
 'truck': 10384}

In [5]:
def get_category_name2id(path_to_labels, num_classes):
    label_map = label_map_util.load_labelmap(path_to_labels)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return {item['name']: item['id'] for item in category_index.values()}

NUM_CLASSES = 9
PATH_TO_LABELS = "/root/models/research/object_detection/data/AI_VMS_label_map.pbtxt"

category_name2id = get_category_name2id(PATH_TO_LABELS, NUM_CLASSES)  

In [6]:
imdb_group.dumpTFRecord("/root/data/tmp_data/AI_VMS_rc1.record", category_name2id)

Tensorflow Record has been writen into /root/data/tmp_data/AI_VMS_rc1.record
